# **Data Generation MIMESIS**

In [ ]:
!pip install kafka-python -q
!pip install mimesis -q
!pip install fastavro -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.1 MB/s eta 0:00:00


In [ ]:
!pip install azure-eventhub -q
!pip install azure-eventhub-kafka -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 16.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement azure-eventhub-kafka (from versions: none)
ERROR: No matching distribution found for azure-eventhub-kafka


In [ ]:
!pip install azure-eventhub-checkpointstoreblob -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.6/353.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
from azure.eventhub import EventHubProducerClient, EventData
from azure.eventhub.aio import EventHubProducerClient as EventHubProducerClientAsync
from azure.eventhub.extensions.checkpointstoreblob import BlobCheckpointStore
from mimesis import Person, Address, Datetime
import uuid, json, random, time, fastavro, io, datetime
import numpy as np

with open('/content/ride_request_schema.json') as f:
    ride_schema = json.load(f)

EVENT_HUB_NAMESPACE = "iesstsabdbaa-grp-05-08"
EVENT_HUB_NAME = "ride_request_group7"
CONNECTION_STRING = "Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc="
producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING, eventhub_name=EVENT_HUB_NAME)


def generate_ride_request():
    address = Address()

    # Weighted choices for status (more completed/accepted than cancelled)
    status = random.choices(
        ["requested", "cancelled", "accepted", "completed"],
        weights=[0.2, 0.1, 0.3, 0.4],  # 40% completed, 30% accepted, etc.
        k=1
    )[0]

    # Weighted choices for ride type (more standard than others)
    ride_type = random.choices(
        ["standard", "premium", "pool"],
        weights=[0.7, 0.2, 0.1],  # 70% standard, 20% premium, 10% pool
        k=1
    )[0]

    # Base price based on ride type
    if ride_type == "premium":
        base_price = random.uniform(15, 50)  # Higher range for premium
    elif ride_type == "pool":
        base_price = random.uniform(5, 15)   # Lower range for pool
    else:
        base_price = random.uniform(8, 25)   # Standard range

    # Distance with normal distribution (skewed toward 4-6 km)
    distance = np.random.normal(loc=5, scale=2)  # Mean=5, std=2
    distance = max(1, min(20, distance))  # Clamp between 1-20 km

    # Adjust price slightly based on distance
    price = base_price + (distance * 0.5)

    # Demand level based on time of day (simplified)
    hour = datetime.datetime.now().hour
    demand_level = "high" if (7 <= hour < 10) or (16 <= hour < 19) else "low"

    return {
        "request_id": str(uuid.uuid4()),
        "passenger_id": str(uuid.uuid4()),
        "pickup_lat": float(address.latitude()),
        "pickup_lng": float(address.longitude()),
        "destination_lat": float(address.latitude()),
        "destination_lng": float(address.longitude()),
        "ride_type": ride_type,
        "status": status,
        "estimate_price": round(price, 2),
        "demand_level": demand_level,
        "estimated_distance_km": round(distance, 2),
        "datetime": datetime.datetime.utcnow().isoformat()
    }
print(f"Producing ride requests to {EVENT_HUB_NAME}...")
try:
    while True:
        data = generate_ride_request()
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(json.dumps(data)))
        producer.send_batch(event_data_batch)
        print(f"Sent: {data['request_id']} at {data['datetime']}")
        time.sleep(random.uniform(0.1, 0.2))
except KeyboardInterrupt:
    print("Stopping producer...")
finally:
    producer.close()

Streaming output truncated to the last 5000 lines.
Sent: 38ebde35-fc85-448e-9aa4-e2b76781d280 at 2025-04-21T19:22:14.236403
Sent: a50475eb-f310-4980-bed5-420d91800646 at 2025-04-21T19:22:14.495875
Sent: ad2e770e-c36c-4578-99fd-4af93f764d1f at 2025-04-21T19:22:14.828968
Sent: fa09cbca-31cd-4994-a24b-ab007bf0b290 at 2025-04-21T19:22:15.165575
Sent: 3f00c58e-91da-420d-b035-427f888a92ac at 2025-04-21T19:22:15.451053
Sent: fa7e7c90-f282-4e70-bbca-2b3ead3eda17 at 2025-04-21T19:22:15.742629
Sent: 3d405734-c640-45b3-adaa-7c0ec0937de1 at 2025-04-21T19:22:16.030211
Sent: c24dc899-ca83-4872-a722-9a16cd646a5f at 2025-04-21T19:22:16.379834
Sent: 251208c3-35f2-4723-81e9-20d8a6cdf795 at 2025-04-21T19:22:16.636386
Sent: e96fbe50-47e1-4813-8c1e-263846f7f26f at 2025-04-21T19:22:16.905954
Sent: ba3d10f1-c855-45da-81cf-2907bdacdc86 at 2025-04-21T19:22:17.181706
Sent: e0601b63-c26f-4b6a-a18c-6890a1ad58ef at 2025-04-21T19:22:17.557420
Sent: c42cfc48-5d02-488c-84a0-d089b9f03ba4 at 2025-04-21T19:22:17.888893


In [ ]:
with open('/content/driver_status_schema.json') as f:
    ride_schema = json.load(f)

EVENT_HUB_NAMESPACE = "iesstsabdbaa-grp-05-08"
EVENT_HUB_NAME = "driver_status_group7"
CONNECTION_STRING = "Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc="

producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING, eventhub_name=EVENT_HUB_NAME)
def generate_driver_status():
    address = Address()

    # Weighted status distribution (more available and on_trip than offline)
    status = random.choices(
        ["available", "en_route", "on_trip", "offline"],
        weights=[0.5, 0.3, 0.15, 0.05],  # 40% available, 30% on_trip, 20% en_route, 10% offline
        k=1
    )[0]

    # Weighted vehicle type distribution (Uber most common, motorbike least common)
    vehicle_type = random.choices(
        ["uber", "cabify", "lyft", "SUV", "sedan", "motorbike"],
        weights=[0.37, 0.23, 0.11, 0.11, 0.12, 0.06],  # Uber 35%, Cabify 25%, etc.
        k=1
    )[0]

    return {
        "driver_id": str(uuid.uuid4()),
        "current_lat": float(address.latitude()),
        "current_lng": float(address.longitude()),
        "status": status,
        "vehicle_type": vehicle_type,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }

print(f"Producing status requests to {EVENT_HUB_NAME}...")
try:
    while True:
        data = generate_driver_status()
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(json.dumps(data)))
        producer.send_batch(event_data_batch)
        print(f"Sent: {data['driver_id']} ({data['status']}) at {data['timestamp']}")
        time.sleep(random.uniform(0.1, 0.15))
except KeyboardInterrupt:
    print("Stopping producer...")
finally:
    producer.close()

Streaming output truncated to the last 5000 lines.
Sent: 2de0fa36-ec98-44d1-92bd-3b0e6e885829 (en_route) at 2025-04-21T19:49:35.263504
Sent: bd31fc41-3027-4750-9ee2-7085d9303050 (en_route) at 2025-04-21T19:49:35.528379
Sent: dabd5dae-5931-4415-84cb-deaf0139efae (en_route) at 2025-04-21T19:49:35.811869
Sent: ed08437c-7889-4c2a-b427-45d9e47196d0 (available) at 2025-04-21T19:49:36.076639
Sent: b2a0a632-d8bc-41cd-a2b2-5c1f4c3bc8ba (en_route) at 2025-04-21T19:49:36.329911
Sent: 123fe106-a753-43ab-bd2d-8749a95823ed (on_trip) at 2025-04-21T19:49:36.625995
Sent: 4f60dc4b-d166-4903-ba00-76d9d24ebda1 (available) at 2025-04-21T19:49:36.922897
Sent: 10c8a333-1b14-4d91-a2ab-3825bd1cafa1 (on_trip) at 2025-04-21T19:49:37.212598
Sent: dd669628-0e57-418a-9467-41d027dcaaa4 (available) at 2025-04-21T19:49:37.504144
Sent: d3c80dd9-75b1-4362-98ff-e840c2c2c9bc (en_route) at 2025-04-21T19:49:37.767831
Sent: 4eb25cb5-7ed2-4e41-ab5d-cb4a71780361 (available) at 2025-04-21T19:49:38.058595
Sent: eba394a3-9eee-4c9

# **Spark**

In [ ]:
# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download Spark 3.4.1 with Hadoop 3
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Unpack Spark
!tar -xvzf spark-3.4.1-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark


spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaSparkIntegration") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .getOrCreate()


In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, FloatType
import time
from threading import Thread, Event
import os

# ==============================================
# 1. INITIALIZATION AND CONFIGURATION
# ==============================================

# Blob Storage Credentials
spark.conf.set(
    "fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
    "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=="
)

# Stream Optimization Settings
STREAM_CONFIG = {
    "processing_time": "10 seconds",
    "max_offsets": 5000,
    "min_offsets": 1000,
    "run_duration": 1200,  # 20 minutes
    "checkpoint_path": "wasbs://group7@iesstsabdbaa.blob.core.windows.net/new_checkpoints"
}

# Event Hub Configurations
event_hub_namespace = "iesstsabdbaa-grp-05-08"

ride_config = {
    "name": "ride_request_group7",
    "kafka_conf": {
        "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
        "subscribe": "ride_request_group7",
        "kafka.sasl.mechanism": "PLAIN",
        "kafka.security.protocol": "SASL_SSL",
        "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc=;EntityPath=ride_request_group7";'
    }
}

driver_config = {
    "name": "driver_status_group7",
    "kafka_conf": {
        "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
        "subscribe": "driver_status_group7",
        "kafka.sasl.mechanism": "PLAIN",
        "kafka.security.protocol": "SASL_SSL",
        "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc=;EntityPath=driver_status_group7";'
    }
}

# Define Schemas
ride_schema = StructType() \
    .add("request_id", StringType()) \
    .add("passenger_id", StringType()) \
    .add("pickup_lat", FloatType()) \
    .add("pickup_lng", FloatType()) \
    .add("destination_lat", FloatType()) \
    .add("destination_lng", FloatType()) \
    .add("ride_type", StringType()) \
    .add("status", StringType()) \
    .add("estimate_price", FloatType()) \
    .add("demand_level", StringType()) \
    .add("estimated_distance_km", FloatType()) \
    .add("datetime", StringType())

driver_schema = StructType() \
    .add("driver_id", StringType()) \
    .add("current_lat", FloatType()) \
    .add("current_lng", FloatType()) \
    .add("status", StringType()) \
    .add("vehicle_type", StringType()) \
    .add("timestamp", StringType())

# ==============================================
# 2. STREAM INITIALIZATION
# ==============================================

def init_optimized_stream(source_config, schema, stream_name):
    df = (spark.readStream
          .format("kafka")
          .options(**source_config["kafka_conf"])
          .option("startingOffsets", "earliest")
          .load()
          .selectExpr("CAST(value AS STRING) as json_str")
          .select(from_json(col("json_str"), schema).alias("data"))
          .select("data.*"))

    return (df.writeStream
            .format("parquet")
            .option("path", f"wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_{stream_name}_data")
            .option("checkpointLocation", f"{STREAM_CONFIG['checkpoint_path']}/{stream_name}")
            .trigger(processingTime=STREAM_CONFIG["processing_time"])
            .option("maxOffsetsPerTrigger", STREAM_CONFIG["max_offsets"])
            .option("minOffsetsPerTrigger", STREAM_CONFIG["min_offsets"])
            .outputMode("append")
            .queryName(f"optimized_{stream_name}"))

# Start both streams
ride_stream = init_optimized_stream(ride_config, ride_schema, "ride").start()
driver_stream = init_optimized_stream(driver_config, driver_schema, "driver").start()

# ==============================================
# 3. MONITORING SYSTEM
# ==============================================

def monitor():
    start = time.time()
    while time.time() - start < STREAM_CONFIG["run_duration"]:
        time.sleep(30)
        print("\n=== Stream Progress ===")
        for stream in [ride_stream, driver_stream]:
            status = stream.status
            progress = stream.lastProgress or {}
            print(f"\n{stream.name}:")
            print(f"Status: {status['message']}")
            print(f"Records processed: {progress.get('numInputRows', 0)}")
            print(f"Processing rate: {progress.get('inputRowsPerSecond', 0):.1f} rows/sec")
            print(f"Latest offsets: {progress.get('sources', [{}])[0].get('endOffset', 'N/A')}")

# Start monitoring thread
monitor_thread = Thread(target=monitor)
monitor_thread.daemon = True
monitor_thread.start()

# ==============================================
# 4. SHUTDOWN HANDLING
# ==============================================

try:
    monitor_thread.join(STREAM_CONFIG["run_duration"])
except KeyboardInterrupt:
    print("\nManual shutdown requested")

print("\n=== Stopping Streams ===")
ride_stream.stop()
driver_stream.stop()

# ==============================================
# 5. FINAL VERIFICATION
# ==============================================

print("\n=== Final Data Verification ===")
for name, path in [("Ride", "optimized_ride_data"), ("Driver", "optimized_driver_data")]:
    try:
        full_path = f"wasbs://group7@iesstsabdbaa.blob.core.windows.net/{path}/*"
        df = spark.read.parquet(full_path)
        count = df.count()
        print(f"\n{name} records: {count}")
        if count > 0:
            df.show(5, truncate=False)
        print(f"Storage location: {full_path}")
    except Exception as e:
        print(f"\nError reading {name} data: {str(e)}")

spark.stop()
print("\nProcessing complete!")


=== Stream Progress ===

optimized_ride:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

=== Stream Progress ===

optimized_ride:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

=== Stream Progress ===

optimized_ride:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

=== Stream Progress ===

optimized_ride:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rat

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.4.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.4.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

# 1. Initialize Spark (if not already running)
spark = SparkSession.builder \
    .appName("RideAnalysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net", "your_access_key_here") \
    .getOrCreate()

# 2. Read stored data
def analyze_data():
    try:
        print("Reading ride data...")
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*")

        print("Reading driver data...")
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*")

        # 3. Perform analysis
        print("\n=== Ride Analysis ===")
        ride_df.groupBy("status").count().show()
        ride_df.groupBy("ride_type").avg("estimate_price").show()

        print("\n=== Driver Analysis ===")
        driver_df.groupBy("status").count().show()
        driver_df.groupBy("vehicle_type").count().show()

        # 4. Sample visualization data
        print("\nTop 5 Rides:")
        ride_df.show(5)

        print("\nTop 5 Drivers:")
        driver_df.show(5)

        return ride_df, driver_df

    except Exception as e:
        print(f"Analysis error: {str(e)}")
        return None, None

# Run analysis
ride_df, driver_df = analyze_data()

# 5. Keep Spark running for further queries
# spark.stop()  # Only run this when completely done

Reading ride data...
Reading driver data...

=== Ride Analysis ===
+---------+-----+
|   status|count|
+---------+-----+
|completed| 3788|
|cancelled| 2216|
|requested| 2629|
| accepted| 3180|
|     null|60220|
+---------+-----+

+---------+-------------------+
|ride_type|avg(estimate_price)|
+---------+-------------------+
|     pool| 24.996760093041843|
| standard|  22.28900705265847|
|  premium| 30.020964794684698|
|     null|               null|
+---------+-------------------+


=== Driver Analysis ===

=== Stream Progress ===

optimized_ride:
Status: Stopped
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: {'ride_request_group7': {'2': 18008, '1': 18009, '3': 18008, '0': 18008}}

optimized_driver:
Status: Stopped
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: {'driver_status_group7': {'2': 16179, '1': 16178, '3': 16178, '0': 16178}}
+---------+-----+
|   status|count|
+---------+-----+
|     null|50251|
|  offline| 2547|
|  on_trip| 3090|
|ava

# **Dashboard example**
Redo it using streamlit

In [ ]:
# First install required packages
!pip install pyspark==3.4.1 pandas matplotlib ipywidgets
!pip install --upgrade pyspark[hadoop]

  Using cached pyspark-3.4.1-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-connect 0.5.2 requires pyspark>=3.5, but you have pyspark 3.4.1 which is incompatible.
  Using cached pyspark-3.5.5-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.1
    Uninstalling pyspark-3.4.1:
      Successfully uninstalled pyspark-3.4.1


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AzureBlobDashboard") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Verify Azure connectivity
try:
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem \
        .get(spark.sparkContext._jsc.hadoopConfiguration()) \
        .listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path(
            "wasbs://group7@iesstsabdbaa.blob.core.windows.net/"
        ))
    print("✅ Successfully connected to Azure Blob Storage")
    print(f"Found {len(files)} items in container")
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

# Create widgets
refresh_button = widgets.Button(description="Refresh Data", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Filter Status:',
    style={'description_width': 'initial'}
)
output = widgets.Output()

# Load data function with error handling
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*")
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*")
        return ride_df.toPandas(), driver_df.toPandas()
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Update dashboard function
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        print("🔄 Loading data...")

        try:
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Filter by status
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Create dashboard layout
            display(widgets.HTML("<h2>🚖 Ride Sharing Analytics Dashboard</h2>"))

            # Metrics row
            metrics_html = f"""
            <div style="display: flex; justify-content: space-between; margin: 10px 0;">
                <div class="metric-card">📊 Total Rides: <b>{len(ride_df)}</b></div>
                <div class="metric-card">🚕 Active Drivers: <b>{len(driver_df[driver_df['status'] != 'offline'])}</b></div>
                <div class="metric-card">💰 Avg Price: <b>${ride_df['estimate_price'].mean():.2f}</b></div>
                <div class="metric-card">📅 Last Updated: <b>{time.strftime('%H:%M:%S')}</b></div>
            </div>
            """
            display(widgets.HTML(metrics_html))

            # Visualizations
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))

            # Plot 1: Ride Type Distribution
            ride_df['ride_type'].value_counts().plot(
                kind='pie',
                ax=axes[0],
                autopct='%1.1f%%',
                colors=['#4CAF50', '#2196F3', '#FFC107']
            )
            axes[0].set_title('Ride Types', pad=20)

            # Plot 2: Status Distribution
            status_counts = ride_df['status'].value_counts()
            status_counts.plot(
                kind='bar',
                ax=axes[1],
                color=['#4CAF50', '#2196F3', '#FFC107', '#F44336']
            )
            axes[1].set_title('Ride Status', pad=20)
            axes[1].tick_params(axis='x', rotation=45)

            # Plot 3: Vehicle Types
            driver_df['vehicle_type'].value_counts().plot(
                kind='bar',
                ax=axes[2],
                color=['#4CAF50', '#2196F3', '#FFC107', '#9C27B0']
            )
            axes[2].set_title('Driver Vehicles', pad=20)
            axes[2].tick_params(axis='x', rotation=45)

            plt.tight_layout()
            plt.show()

            # Data preview
            display(widgets.HTML("<h4>Sample Ride Data</h4>"))
            display(ride_df.head(3))

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')

# Initial layout
dashboard = widgets.VBox([
    widgets.HBox([refresh_button, status_dropdown]),
    output
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option (uncomment if needed)
# import threading
# def auto_refresh():
#     while True:
#         time.sleep(30)
#         refresh_button.click()
# threading.Thread(target=auto_refresh, daemon=True).start()

❌ Connection failed: Wrong FS: wasbs://group7@iesstsabdbaa.blob.core.windows.net/, expected: file:///


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder \
    .appName("EnhancedRideAnalytics") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Verify Azure connectivity
try:
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem \
        .get(spark.sparkContext._jsc.hadoopConfiguration()) \
        .listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path(
            "wasbs://group7@iesstsabdbaa.blob.core.windows.net/"
        ))
    print("✅ Successfully connected to Azure Blob Storage")
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time
import folium
from datetime import datetime

# Install folium if needed
try:
    from folium import plugins
except:
    !pip install folium
    from folium import plugins

# Create widgets
refresh_button = widgets.Button(description="Refresh Dashboard", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Ride Status:',
    style={'description_width': 'initial'}
)
time_window = widgets.Dropdown(
    options=['Last Hour', 'Today', 'Last 7 Days', 'All Time'],
    value='Today',
    description='Time Window:',
    style={'description_width': 'initial'}
)
analysis_tabs = widgets.Tab()
output = widgets.Output()

# Load data function with enhanced time filtering
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*").toPandas()
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*").toPandas()

        # Convert timestamps
        ride_df['datetime'] = pd.to_datetime(ride_df['datetime'])
        driver_df['timestamp'] = pd.to_datetime(driver_df['timestamp'])

        return ride_df, driver_df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Enhanced KPI calculations
def calculate_kpis(ride_df, driver_df):
    kpis = {}

    # Basic Metrics
    kpis['total_rides'] = len(ride_df)
    kpis['active_drivers'] = len(driver_df[driver_df['status'] != 'offline'])
    kpis['avg_price'] = ride_df['estimate_price'].mean()

    # Revenue Analysis
    kpis['total_revenue'] = ride_df['estimate_price'].sum()
    kpis['revenue_per_ride'] = kpis['total_revenue'] / kpis['total_rides'] if kpis['total_rides'] > 0 else 0

    # Time-Based Metrics
    time_diff = (ride_df['datetime'].max() - ride_df['datetime'].min()).total_seconds() / 3600
    kpis['rides_per_hour'] = kpis['total_rides'] / max(1, time_diff)

    # Driver Utilization
    kpis['driver_utilization'] = (kpis['active_drivers'] / len(driver_df)) * 100 if len(driver_df) > 0 else 0

    # Cancellation Analysis
    kpis['cancellation_rate'] = (len(ride_df[ride_df['status'] == 'cancelled']) / kpis['total_rides']) * 100 if kpis['total_rides'] > 0 else 0

    return kpis

# Update dashboard function with all new features
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        start_time = time.time()

        try:
            # Load data
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Apply filters
            current_time = datetime.now()
            time_filter = {
                'Last Hour': pd.Timedelta(hours=1),
                'Today': pd.Timedelta(days=1),
                'Last 7 Days': pd.Timedelta(days=7),
                'All Time': pd.Timedelta(days=3650)  # 10 years
            }[time_window.value]

            ride_df = ride_df[ride_df['datetime'] >= (current_time - time_filter)]
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Calculate KPIs
            kpis = calculate_kpis(ride_df, driver_df)

            # ====================
            # Dashboard Layout
            # ====================

            # Header
            display(HTML("<h1 style='color: #1a73e8;'>🚖 Advanced Ride Analytics Dashboard</h1>"))
            display(HTML(f"<p>Last Updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')} | Processing Time: {(time.time()-start_time):.2f}s</p>"))

            # ====================
            # KPI Metrics Cards
            # ====================
            metrics_html = f"""
            <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 10px; margin: 15px 0;">
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>💰 Revenue</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>${kpis['total_revenue']:,.2f}</p>
                    <p style='margin: 0; font-size: 14px;'>${kpis['revenue_per_ride']:.2f} per ride</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>📊 Rides</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['total_rides']:,}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['rides_per_hour']:.1f} rides/hour</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>🚕 Drivers</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['active_drivers']}/{len(driver_df)}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['driver_utilization']:.1f}% utilization</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>⚠️ Cancellations</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['cancellation_rate']:.1f}%</p>
                    <p style='margin: 0; font-size: 14px;'>Avg price: ${ride_df[ride_df['status']=='cancelled']['estimate_price'].mean():.2f}</p>
                </div>
            </div>
            """
            display(HTML(metrics_html))

            # ====================
            # Main Visualization Tabs
            # ====================
            tab_outputs = []

            # Tab 1: Time Analysis
            with widgets.Output() as tab1:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                # Hourly Demand
                hourly = ride_df.set_index('datetime').resample('H')['request_id'].count()
                hourly.plot(ax=ax1, title='Hourly Ride Demand', color='#1a73e8')
                ax1.set_ylabel("Number of Rides")
                ax1.grid(True, alpha=0.3)

                # Price Distribution
                ride_df['estimate_price'].plot(kind='box', ax=ax2, vert=False,
                                             patch_artist=True,
                                             boxprops=dict(facecolor='#1a73e8'))
                ax2.set_title('Ride Price Distribution')
                ax2.set_xlabel("Price ($)")
                ax2.grid(True, alpha=0.3)

                plt.tight_layout()
                plt.show()

                # Weekday Analysis
                if time_window.value in ['Last 7 Days', 'All Time']:
                    fig, ax = plt.subplots(figsize=(10, 4))
                    ride_df['weekday'] = ride_df['datetime'].dt.day_name()
                    ride_df['weekday'].value_counts().reindex([
                        'Monday', 'Tuesday', 'Wednesday',
                        'Thursday', 'Friday', 'Saturday', 'Sunday'
                    ]).plot(kind='bar', ax=ax, color='#1a73e8')
                    ax.set_title('Rides by Weekday')
                    ax.set_ylabel("Number of Rides")
                    plt.tight_layout()
                    plt.show()

            # Tab 2: Geospatial Analysis
            with widgets.Output() as tab2:
                # Create base map
                avg_lat = ride_df['pickup_lat'].mean()
                avg_lng = ride_df['pickup_lng'].mean()
                m = folium.Map(location=[avg_lat, avg_lng], zoom_start=12)

                # Add heatmap
                heat_data = [[row['pickup_lat'], row['pickup_lng']]
                            for _, row in ride_df.iterrows()]
                plugins.HeatMap(heat_data, radius=15).add_to(m)

                # Add cluster markers
                marker_cluster = plugins.MarkerCluster().add_to(m)
                for _, row in ride_df.sample(min(100, len(ride_df))).iterrows():
                    folium.Marker(
                        [row['pickup_lat'], row['pickup_lng']],
                        popup=f"${row['estimate_price']} {row['ride_type']}",
                        icon=folium.Icon(color='blue' if row['status']=='completed' else 'red')
                    ).add_to(marker_cluster)

                # Display map
                display(m)

                # Price by Area analysis
                ride_df['pickup_area'] = pd.cut(ride_df['pickup_lat'], bins=5, labels=[
                    'South', 'Mid-South', 'Central', 'Mid-North', 'North'
                ])
                area_prices = ride_df.groupby('pickup_area')['estimate_price'].mean()
                plt.figure(figsize=(10, 4))
                area_prices.plot(kind='bar', color='#1a73e8')
                plt.title('Average Price by Area')
                plt.ylabel("Price ($)")
                plt.tight_layout()
                plt.show()

            # Tab 3: Driver-Ride Analysis
            with widgets.Output() as tab3:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                # Driver Status
                driver_df['status'].value_counts().plot(
                    kind='pie', ax=ax1, autopct='%1.1f%%',
                    colors=['#34a853', '#1a73e8', '#fbbc05', '#ea4335']
                )
                ax1.set_title('Driver Status Distribution')

                # Vehicle Type Analysis
                vehicle_analysis = driver_df.groupby('vehicle_type').agg({
                    'driver_id': 'count',
                    'current_lat': 'mean'  # Just for demo
                })
                vehicle_analysis['driver_id'].plot(
                    kind='bar', ax=ax2, color='#1a73e8'
                )
                ax2.set_title('Drivers by Vehicle Type')
                ax2.set_ylabel("Number of Drivers")

                plt.tight_layout()
                plt.show()

                # Driver-Ride Correlation
                if not ride_df.empty and not driver_df.empty:
                    plt.figure(figsize=(10, 4))
                    hourly_drivers = driver_df.set_index('timestamp').resample('H')['driver_id'].count()
                    hourly_rides = ride_df.set_index('datetime').resample('H')['request_id'].count()
                    pd.DataFrame({
                        'Drivers': hourly_drivers,
                        'Rides': hourly_rides
                    }).plot(title='Drivers vs Rides (Hourly)')
                    plt.ylabel("Count")
                    plt.tight_layout()
                    plt.show()

            # Create tabs
            analysis_tabs.children = [tab1, tab2, tab3]
            analysis_tabs.titles = ['Time Analysis', 'Geospatial', 'Driver Insights']
            display(analysis_tabs)

            # ====================
            # Data Preview Section
            # ====================
            display(HTML("<h3 style='margin-top: 20px;'>📋 Data Preview</h3>"))

            # Create tabs for data preview
            preview_tabs = widgets.Tab()
            with widgets.Output() as preview1:
                display(HTML("<h4>Ride Data Sample</h4>"))
                display(ride_df[['datetime', 'ride_type', 'status', 'estimate_price']].head(5))

            with widgets.Output() as preview2:
                display(HTML("<h4>Driver Data Sample</h4>"))
                display(driver_df[['timestamp', 'status', 'vehicle_type']].head(5))

            preview_tabs.children = [preview1, preview2]
            preview_tabs.titles = ['Rides', 'Drivers']
            display(preview_tabs)

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")
            import traceback
            traceback.print_exc()

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')
time_window.observe(update_dashboard, names='value')

# Dashboard layout
controls = widgets.HBox([refresh_button, status_dropdown, time_window])
dashboard = widgets.VBox([
    controls,
    output
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option
import threading
def auto_refresh():
    while True:
        time.sleep(300)  # Refresh every 5 minutes
        refresh_button.click()

threading.Thread(target=auto_refresh, daemon=True).start()

❌ Connection failed: Wrong FS: wasbs://group7@iesstsabdbaa.blob.core.windows.net/, expected: file:///


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder \
    .appName("EnhancedRideAnalytics") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time
import folium
from datetime import datetime

# Install folium if needed
try:
    from folium import plugins
except:
    !pip install folium
    from folium import plugins

# Create widgets
refresh_button = widgets.Button(description="Refresh Dashboard", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Ride Status:',
    style={'description_width': 'initial'}
)
time_window = widgets.Dropdown(
    options=['Last Hour', 'Today', 'Last 7 Days', 'All Time'],
    value='Today',
    description='Time Window:',
    style={'description_width': 'initial'}
)

# Initialize tabs properly
output = widgets.Output()
analysis_tabs = widgets.Tab()
tab1_output = widgets.Output()
tab2_output = widgets.Output()
tab3_output = widgets.Output()
analysis_tabs.children = [tab1_output, tab2_output, tab3_output]
analysis_tabs.titles = ['Time Analysis', 'Geospatial', 'Driver Insights']

# Load data function with enhanced time filtering
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*").toPandas()
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*").toPandas()

        # Convert timestamps
        ride_df['datetime'] = pd.to_datetime(ride_df['datetime'])
        driver_df['timestamp'] = pd.to_datetime(driver_df['timestamp'])

        return ride_df, driver_df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Enhanced KPI calculations
def calculate_kpis(ride_df, driver_df):
    kpis = {}

    # Basic Metrics
    kpis['total_rides'] = len(ride_df)
    kpis['active_drivers'] = len(driver_df[driver_df['status'] != 'offline'])
    kpis['avg_price'] = ride_df['estimate_price'].mean()

    # Revenue Analysis
    kpis['total_revenue'] = ride_df['estimate_price'].sum()
    kpis['revenue_per_ride'] = kpis['total_revenue'] / kpis['total_rides'] if kpis['total_rides'] > 0 else 0

    # Time-Based Metrics
    time_diff = (ride_df['datetime'].max() - ride_df['datetime'].min()).total_seconds() / 3600
    kpis['rides_per_hour'] = kpis['total_rides'] / max(1, time_diff)

    # Driver Utilization
    kpis['driver_utilization'] = (kpis['active_drivers'] / len(driver_df)) * 100 if len(driver_df) > 0 else 0

    # Cancellation Analysis
    kpis['cancellation_rate'] = (len(ride_df[ride_df['status'] == 'cancelled']) / kpis['total_rides']) * 100 if kpis['total_rides'] > 0 else 0

    return kpis

# Update dashboard function with all new features
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        start_time = time.time()

        try:
            # Load data
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Apply filters
            current_time = datetime.now()
            time_filter = {
                'Last Hour': pd.Timedelta(hours=1),
                'Today': pd.Timedelta(days=1),
                'Last 7 Days': pd.Timedelta(days=7),
                'All Time': pd.Timedelta(days=3650)  # 10 years
            }[time_window.value]

            ride_df = ride_df[ride_df['datetime'] >= (current_time - time_filter)]
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Calculate KPIs
            kpis = calculate_kpis(ride_df, driver_df)

            # ====================
            # Dashboard Layout
            # ====================

            # Header
            display(HTML("<h1 style='color: #1a73e8;'>🚖 Advanced Ride Analytics Dashboard</h1>"))
            display(HTML(f"<p>Last Updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')} | Processing Time: {(time.time()-start_time):.2f}s</p>"))

            # ====================
            # KPI Metrics Cards
            # ====================
            metrics_html = f"""
            <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 10px; margin: 15px 0;">
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>💰 Revenue</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>${kpis['total_revenue']:,.2f}</p>
                    <p style='margin: 0; font-size: 14px;'>${kpis['revenue_per_ride']:.2f} per ride</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>📊 Rides</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['total_rides']:,}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['rides_per_hour']:.1f} rides/hour</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>🚕 Drivers</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['active_drivers']}/{len(driver_df)}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['driver_utilization']:.1f}% utilization</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>⚠️ Cancellations</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['cancellation_rate']:.1f}%</p>
                    <p style='margin: 0; font-size: 14px;'>Avg price: ${ride_df[ride_df['status']=='cancelled']['estimate_price'].mean():.2f}</p>
                </div>
            </div>
            """
            display(HTML(metrics_html))

            # ====================
            # Main Visualization Tabs
            # ====================

            # Clear previous tab contents
            for tab in analysis_tabs.children:
                tab.clear_output()

            # Tab 1: Time Analysis
            with tab1_output:
                try:
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                    # Hourly Demand
                    hourly = ride_df.set_index('datetime').resample('H')['request_id'].count()
                    hourly.plot(ax=ax1, title='Hourly Ride Demand', color='#1a73e8')
                    ax1.set_ylabel("Number of Rides")
                    ax1.grid(True, alpha=0.3)

                    # Price Distribution
                    ride_df['estimate_price'].plot(kind='box', ax=ax2, vert=False,
                                                 patch_artist=True,
                                                 boxprops=dict(facecolor='#1a73e8'))
                    ax2.set_title('Ride Price Distribution')
                    ax2.set_xlabel("Price ($)")
                    ax2.grid(True, alpha=0.3)

                    plt.tight_layout()
                    plt.show()

                    # Weekday Analysis
                    if time_window.value in ['Last 7 Days', 'All Time']:
                        fig, ax = plt.subplots(figsize=(10, 4))
                        ride_df['weekday'] = ride_df['datetime'].dt.day_name()
                        ride_df['weekday'].value_counts().reindex([
                            'Monday', 'Tuesday', 'Wednesday',
                            'Thursday', 'Friday', 'Saturday', 'Sunday'
                        ]).plot(kind='bar', ax=ax, color='#1a73e8')
                        ax.set_title('Rides by Weekday')
                        ax.set_ylabel("Number of Rides")
                        plt.tight_layout()
                        plt.show()
                except Exception as e:
                    print(f"⚠️ Time analysis error: {str(e)}")

            # Tab 2: Geospatial Analysis
            with tab2_output:
                try:
                    # Create base map
                    avg_lat = ride_df['pickup_lat'].mean()
                    avg_lng = ride_df['pickup_lng'].mean()
                    m = folium.Map(location=[avg_lat, avg_lng], zoom_start=12)

                    # Add heatmap
                    heat_data = [[row['pickup_lat'], row['pickup_lng']]
                                for _, row in ride_df.iterrows()]
                    plugins.HeatMap(heat_data, radius=15).add_to(m)

                    # Add cluster markers
                    marker_cluster = plugins.MarkerCluster().add_to(m)
                    for _, row in ride_df.sample(min(100, len(ride_df))).iterrows():
                        folium.Marker(
                            [row['pickup_lat'], row['pickup_lng']],
                            popup=f"${row['estimate_price']} {row['ride_type']}",
                            icon=folium.Icon(color='blue' if row['status']=='completed' else 'red')
                        ).add_to(marker_cluster)

                    # Display map
                    display(m)

                    # Price by Area analysis
                    plt.figure(figsize=(10, 4))
                    ride_df['pickup_area'] = pd.cut(ride_df['pickup_lat'], bins=5, labels=[
                        'South', 'Mid-South', 'Central', 'Mid-North', 'North'
                    ])
                    area_prices = ride_df.groupby('pickup_area')['estimate_price'].mean()
                    area_prices.plot(kind='bar', color='#1a73e8')
                    plt.title('Average Price by Area')
                    plt.ylabel("Price ($)")
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"⚠️ Geospatial error: {str(e)}")

            # Tab 3: Driver-Ride Analysis
            with tab3_output:
                try:
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                    # Driver Status
                    driver_df['status'].value_counts().plot(
                        kind='pie', ax=ax1, autopct='%1.1f%%',
                        colors=['#34a853', '#1a73e8', '#fbbc05', '#ea4335']
                    )
                    ax1.set_title('Driver Status Distribution')

                    # Vehicle Type Analysis
                    vehicle_analysis = driver_df.groupby('vehicle_type').agg({
                        'driver_id': 'count',
                        'current_lat': 'mean'  # Just for demo
                    })
                    vehicle_analysis['driver_id'].plot(
                        kind='bar', ax=ax2, color='#1a73e8'
                    )
                    ax2.set_title('Drivers by Vehicle Type')
                    ax2.set_ylabel("Number of Drivers")

                    plt.tight_layout()
                    plt.show()

                    # Driver-Ride Correlation
                    if not ride_df.empty and not driver_df.empty:
                        plt.figure(figsize=(10, 4))
                        hourly_drivers = driver_df.set_index('timestamp').resample('H')['driver_id'].count()
                        hourly_rides = ride_df.set_index('datetime').resample('H')['request_id'].count()
                        pd.DataFrame({
                            'Drivers': hourly_drivers,
                            'Rides': hourly_rides
                        }).plot(title='Drivers vs Rides (Hourly)')
                        plt.ylabel("Count")
                        plt.tight_layout()
                        plt.show()
                except Exception as e:
                    print(f"⚠️ Driver analysis error: {str(e)}")

            # Display tabs
            display(analysis_tabs)

            # ====================
            # Data Preview Section
            # ====================
            display(HTML("<h3 style='margin-top: 20px;'>📋 Data Preview</h3>"))

            # Create tabs for data preview
            preview_tabs = widgets.Tab()
            with widgets.Output() as preview1:
                display(HTML("<h4>Ride Data Sample</h4>"))
                display(ride_df[['datetime', 'ride_type', 'status', 'estimate_price']].head(5))

            with widgets.Output() as preview2:
                display(HTML("<h4>Driver Data Sample</h4>"))
                display(driver_df[['timestamp', 'status', 'vehicle_type']].head(5))

            preview_tabs.children = [preview1, preview2]
            preview_tabs.titles = ['Rides', 'Drivers']
            display(preview_tabs)

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')
time_window.observe(update_dashboard, names='value')

# Dashboard layout
controls = widgets.HBox([refresh_button, status_dropdown, time_window])
dashboard = widgets.VBox([
    controls,
    output,
    analysis_tabs
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option
import threading
def auto_refresh():
    while True:
        time.sleep(300)  # Refresh every 5 minutes
        refresh_button.click()

threading.Thread(target=auto_refresh, daemon=True).start()

#Opening the Spark Job

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AzureBlobDashboard") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

print("✅ SparkSession connected to Azure Blob Storage")


✅ SparkSession connected to Azure Blob Storage


In [ ]:
# Define paths for both ride and driver data
ride_data_path = "wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*"
driver_data_path = "wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*"

# Load both data feeds
df_ride = spark.read.parquet(ride_data_path)
df_driver = spark.read.parquet(driver_data_path)

# Optional: Create temporary views for SQL queries
df_ride.createOrReplaceTempView("ride_data")
df_driver.createOrReplaceTempView("driver_data")

print("✅ Both data feeds loaded successfully.")

✅ Both data feeds loaded successfully.


In [ ]:
df_ride.printSchema()
df_driver.printSchema()


root
 |-- request_id: string (nullable = true)
 |-- passenger_id: string (nullable = true)
 |-- pickup_lat: float (nullable = true)
 |-- pickup_lng: float (nullable = true)
 |-- destination_lat: float (nullable = true)
 |-- destination_lng: float (nullable = true)
 |-- ride_type: string (nullable = true)
 |-- status: string (nullable = true)
 |-- estimate_price: float (nullable = true)
 |-- demand_level: string (nullable = true)
 |-- estimated_distance_km: float (nullable = true)
 |-- datetime: string (nullable = true)

root
 |-- driver_id: string (nullable = true)
 |-- current_lat: float (nullable = true)
 |-- current_lng: float (nullable = true)
 |-- status: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



#Basic Spark Streaming Analytcis:

In [ ]:
# For ride_df
df_ride = df_ride.na.drop()

 # For driver_df
dr_driver = df_driver.na.drop()

In [ ]:
from pyspark.sql.functions import count, when

df_ride_status = df_ride.agg(
    count("*").alias("total_rides"),
    count(when(df_ride["status"] == "accepted", True)).alias("accepted_rides"),
    count(when(df_ride["status"] == "cancelled", True)).alias("cancelled_rides"),
    count(when(df_ride["status"] == "completed", True)).alias("completed_rides"),
    count(when(df_ride["status"] == "requested", True)).alias("requested_rides")
)

df_ride_status.show()


+-----------+--------------+---------------+---------------+---------------+
|total_rides|accepted_rides|cancelled_rides|completed_rides|requested_rides|
+-----------+--------------+---------------+---------------+---------------+
|      11813|          3180|           2216|           3788|           2629|
+-----------+--------------+---------------+---------------+---------------+



In [ ]:
from pyspark.sql.functions import col

df_ride.select("request_id", "estimated_distance_km", "ride_type").show(5)


+--------------------+---------------------+---------+
|          request_id|estimated_distance_km|ride_type|
+--------------------+---------------------+---------+
|92bbc23d-0f6e-45c...|                19.09| standard|
|02d87f3d-3fdf-457...|                  7.4|  premium|
|953cf8f7-662b-4e2...|                15.98| standard|
|d8a5f163-4129-473...|                14.88| standard|
|2cc85368-bca6-44b...|                 5.45|     pool|
+--------------------+---------------------+---------+
only showing top 5 rows



In [ ]:
df_cancel_rate = df_ride.agg(
    (count(when(col("status") == "cancelled", True)) / count("*")).alias("cancellation_rate")
)

df_cancel_rate.show()


+-----------------+
|cancellation_rate|
+-----------------+
|0.187589943282824|
+-----------------+



In [ ]:
from pyspark.sql.functions import window, to_timestamp

df_ride = df_ride.withColumn("datetime", to_timestamp("datetime"))

df_windowed = df_ride.groupBy(
    window("datetime", "10 minutes")
).count().orderBy("window")

df_windowed.show(truncate=False)


+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2025-04-20 12:40:00, 2025-04-20 12:50:00}|34   |
|{2025-04-20 12:50:00, 2025-04-20 13:00:00}|1939 |
|{2025-04-20 13:00:00, 2025-04-20 13:10:00}|1937 |
|{2025-04-20 13:10:00, 2025-04-20 13:20:00}|1938 |
|{2025-04-20 13:20:00, 2025-04-20 13:30:00}|89   |
|{2025-04-20 15:30:00, 2025-04-20 15:40:00}|2    |
|{2025-04-21 10:50:00, 2025-04-21 11:00:00}|834  |
|{2025-04-21 19:20:00, 2025-04-21 19:30:00}|1582 |
|{2025-04-21 19:30:00, 2025-04-21 19:40:00}|2007 |
|{2025-04-21 19:40:00, 2025-04-21 19:50:00}|1451 |
+------------------------------------------+-----+



#Intermediate Spark Streaming Analytics

In [ ]:
df_demand = df_ride.groupBy("ride_type").agg(count("*").alias("total_requests"))
df_supply = df_driver.groupBy("vehicle_type").agg(count("*").alias("available_drivers"))

df_match = df_demand.join(df_supply, df_demand.ride_type == df_supply.vehicle_type, "outer") \
    .drop("vehicle_type") \
    .fillna(0)

df_match = df_match.withColumn("demand_supply_ratio", col("total_requests") / (col("available_drivers") + 1))
df_match.show()


+---------+--------------+-----------------+-------------------+
|ride_type|total_requests|available_drivers|demand_supply_ratio|
+---------+--------------+-----------------+-------------------+
|     null|             0|            50251|                0.0|
|     null|             0|             2123|                0.0|
|     null|             0|             2819|                0.0|
|     null|             0|             2100|                0.0|
|     null|             0|             1841|                0.0|
|     pool|          2747|                0|             2747.0|
|  premium|          3265|                0|             3265.0|
|     null|             0|             2090|                0.0|
| standard|          5801|                0|             5801.0|
|     null|             0|             3489|                0.0|
+---------+--------------+-----------------+-------------------+



In [ ]:
df_demand_level = df_ride.groupBy("demand_level").count()
df_demand_level.show()


+------------+-----+
|demand_level|count|
+------------+-----+
|         low| 8372|
|        high| 3441|
+------------+-----+



#Advanced Spark Streaming Analytics

In [ ]:
from pyspark.sql.functions import stddev, mean

stats = df_ride.select(mean("estimated_distance_km").alias("mean_dist"),
                       stddev("estimated_distance_km").alias("std_dist")).collect()[0]

mean_dist = stats["mean_dist"]
std_dist = stats["std_dist"]

df_anomalies = df_ride.filter(
    (col("estimated_distance_km") > mean_dist + 3 * std_dist) |
    (col("estimated_distance_km") < mean_dist - 3 * std_dist)
)

df_anomalies.select("request_id", "estimated_distance_km", "ride_type").show()


+----------+---------------------+---------+
|request_id|estimated_distance_km|ride_type|
+----------+---------------------+---------+
+----------+---------------------+---------+



In [ ]:
from pyspark.sql.functions import round

df_hotspots = df_ride.withColumn("pickup_zone_lat", round("pickup_lat", 1)) \
                     .withColumn("pickup_zone_lng", round("pickup_lng", 1)) \
                     .groupBy("pickup_zone_lat", "pickup_zone_lng") \
                     .count().orderBy("count", ascending=False)

df_hotspots.show(5)


+---------------+---------------+-----+
|pickup_zone_lat|pickup_zone_lng|count|
+---------------+---------------+-----+
|           56.1|         -149.2|    2|
|           66.1|         -103.6|    2|
|           60.6|         -104.1|    2|
|          -68.7|         -147.4|    2|
|           56.8|         -122.5|    2|
+---------------+---------------+-----+
only showing top 5 rows



In [ ]:
df_fraud = df_ride.filter((col("estimated_distance_km") < 1.0) & (col("estimate_price") > 100))
df_fraud.select("request_id", "estimated_distance_km", "estimate_price").show()


+----------+---------------------+--------------+
|request_id|estimated_distance_km|estimate_price|
+----------+---------------------+--------------+
+----------+---------------------+--------------+



#Real time Date




In [ ]:
from mimesis import Person, Address, Datetime
from kafka import KafkaProducer
from datetime import datetime as dt
import json
import random
import time
from azure.eventhub import EventHubProducerClient, EventData
from azure.eventhub.aio import EventHubProducerClient as EventHubProducerClientAsync
from azure.eventhub.extensions.checkpointstoreblob import BlobCheckpointStore
from mimesis import Person, Address, Datetime
import uuid, json, random, time, fastavro, io, datetime
from pyspark.sql.functions import col


# Initialize Mimesis data generators
person = Person("en")
address = Address()
date = Datetime()

# Azure Event Hub Kafka-compatible settings

EVENT_HUB_NAMESPACE = "iesstsabdbaa-grp-05-08"

producer = EventHubProducerClient.from_connection_string(
    conn_str= "Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc=",
    eventhub_name= "ride_request_group7"
)

# --- Ride Request Generator ---

def generate_ride_request():
    return {
        "request_id": person.identifier(),
        "passenger_id": person.identifier(),
        "pickup_lat": address.latitude(),
        "pickup_lng": address.longitude(),
        "destination_lat": address.latitude(),
        "destination_lng": address.longitude(),
        "ride_type": random.choice(["Uber", "Lyft", "Cabify"]),
        "status": random.choice(["requested", "active", "completed"]),
        "estimate_price": round(random.uniform(5.0, 50.0), 2),
        "demand_level": random.choice(["low", "high"]),
        "estimated_distance_km": round(random.uniform(1.0, 25.0), 2),
        "datetime": datetime.now().isoformat()
    }
def stream_data(topic="ride-requests", interval=1, count=10):
    for _ in range(count):
        if topic == "ride-requests":
            data = generate_ride_request()
        else:
            continue
            # --- Driver Status Generator ---
            def generate_driver_status():
                return {
                    "driver_id": person.identifier(),
                    "current_lat": address.latitude(),
                    "current_lng": address.longitude(),
                    "status": random.choice(["available", "busy"]),
                    "vehicle_type": random.choice(["Uber", "Lyft", "Cabify"])
                }


        event_data = EventData(json.dumps(data))  # Make sure this wraps the entire dict
        with producer:
            producer.send_batch([event_data])
        print(f"Sent: {data}")
        time.sleep(interval)


while True:
    stream_data(topic="ride-requests", interval=2, count=1)



PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got float.